### Single Stock Analysis Using 3 Factor Risk Model

In [150]:
import datetime as dt
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

tickers = ['IBM']
start = "2000-01-01"
end = "2020-01-01"

# Grab yfinance Data
df_portfolio = yf.download(tickers, start, end).reset_index()

df_portfolio = df_portfolio[["Date", "Adj Close"]].rename(columns={"Date": "date", "Adj Close": "adj_close"})


[*********************100%***********************]  1 of 1 completed


In [151]:
# Computing Nominal Range
data = df_portfolio["adj_close"]
data/data.iloc[0]

0       1.000000
1       0.966056
2       1.000000
3       0.982759
4       0.978448
          ...   
5026    1.736327
5027    1.735427
5028    1.740058
5029    1.708413
5030    1.724235
Name: adj_close, Length: 5031, dtype: float64

In [152]:
# Calculate Log Returns
df_portfolio["return"] = (df_portfolio["adj_close"] /df_portfolio["adj_close"].shift())

df_portfolio["log_return"] = np.log(df_portfolio["return"])

# Total Return Over Time Series
np.e ** df_portfolio["log_return"].sum()

1.7242353152238032

In [153]:
import pandas as pd

# Calculating Stocks Monthly Returns
df_portfolio_monthly  = df_portfolio[["date", "adj_close"]].set_index("date").resample("M").last().pct_change().dropna()

# Load in Fama French Data
df_fama_french = pd.read_csv("data\\Developed_3_Factors.csv")[:390]

# Clean Fama French Data
df_fama_french = df_fama_french.rename(columns={"Unnamed: 0": "date", "Mkt-RF": "market_premium", "SMB": "smb", "HML": "hml", "RF": "rf"})

# Clean Date Format
df_fama_french["month"] = df_fama_french["date"].str[:4] + "-" + df_fama_french["date"].str[4:]
df_fama_french["month"] = pd.to_datetime(df_fama_french["month"])
df_fama_french

,date,market_premium,smb,hml,rf,month
0,199007,0.77,0.53,-0.36,0.68,1990-07-01
1,199008,-10.77,-1.51,0.48,0.66,1990-08-01
2,199009,-11.89,1.32,0.81,0.60,1990-09-01
3,199010,9.35,-7.58,-4.48,0.68,1990-10-01
4,199011,-3.72,1.44,1.13,0.57,1990-11-01
...,...,...,...,...,...,...
385,202208,-4.22,0.42,2.49,0.19,2022-08-01
386,202209,-9.46,-1.66,1.89,0.19,2022-09-01
387,202210,6.79,-2.02,4.41,0.23,2022-10-01
388,202211,7.25,-0.24,-0.28,0.29,2022-11-01


In [154]:
# Calculate Monthly Returns 
df_portfolio["date"] = df_portfolio["date"].sort_values()
df_portfolio["month"] = df_portfolio["date"].apply(lambda x: str(x)[:7])

# Aggregate Monthly Returns
df_monthly_returns = df_portfolio.groupby("month").sum().reset_index()[["month", "log_return"]]
df_monthly_returns["return"] = np.e ** df_monthly_returns["log_return"]
df_monthly_returns = df_monthly_returns[["month", "return", "log_return"]]
df_monthly_returns["return"] = (df_monthly_returns["return"] - 1) * 100

# Clean Date Formatting
df_monthly_returns["month"] = pd.to_datetime(df_monthly_returns["month"])

C:\Users\PCSale123\AppData\Local\Temp\ipykernel_14192\1585041905.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_monthly_returns = df_portfolio.groupby("month").sum().reset_index()[["month", "log_return"]]


In [158]:
df_combined = pd.merge(df_monthly_returns, df_fama_french, how="inner", on="month")[["month", "return", "market_premium", "smb", "hml", "rf"]].copy(deep=True)
df_combined["market_premium"] = pd.to_numeric(df_combined["market_premium"])
df_combined["smb"] = pd.to_numeric(df_combined["smb"])
df_combined["hml"] = pd.to_numeric(df_combined["hml"])
df_combined["rf"] = pd.to_numeric(df_combined["rf"])

## Determine Risk Exposure to 3 Factors

In [159]:
X = df_combined[['market_premium', 'smb', 'hml']]
y = df_combined['return'] - df_combined['rf']
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     48.79
Date:                Fri, 24 Feb 2023   Prob (F-statistic):           1.43e-24
Time:                        17:26:41   Log-Likelihood:                -756.84
No. Observations:                 240   AIC:                             1522.
Df Residuals:                     236   BIC:                             1536.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.2385      0.377      0.

## Generate Expected Return in a Year

In [160]:
rf = df_combined['rf'].mean()
market_premium = df_combined['market_premium'].mean()
size_premium = df_combined['smb'].mean()
value_premium = df_combined['hml'].mean()

expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 
expected_yearly_return = expected_monthly_return * 12
print("Expected yearly return: " + str(expected_yearly_return))

Expected yearly return: 2.950501571963396
